In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torchvision.transforms.functional as VF
from torchvision import transforms

import sys, argparse, os, copy, itertools, glob, datetime
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_fscore_support
from sklearn.datasets import load_svmlight_file
from collections import OrderedDict

/home/wangyh/miniconda3/envs/patho_AI/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [66]:
feature = np.load('../../data/10X_full_slide_features.npy',allow_pickle=True).item()
train = np.load('../../config/data_segmentation_csv/10X_tv_grouping.npy',allow_pickle=True).item()
full = np.load('../../config/data_segmentation_csv/10X_full.npy',allow_pickle=True).item()
train_index = list(train['tv_list'].index)
full_index = list(full['full_list'].index)
test_index = list(set(full_index)-set(train_index))

index_dict = {ind:i for i,ind in enumerate(full_index)}
label_dict = {'L':0,'H':1}

In [80]:
class args():
    def __init__(self):
        self.dataset = 'TCGA_BLCA'
        self.num_classes = 2

    def __getattribute__(self, __name: str):
        return object.__getattribute__(self, __name)
    
args = args()

In [97]:
def get_bag_feats(index, args):
#     if args.dataset == 'TCGA-lung-default':
#         feats_csv_path = 'datasets/tcga-dataset/tcga_lung_data_feats/' + csv_file_df.iloc[0].split('/')[1] + '.csv'
#     else:
#         feats_csv_path = csv_file_df.iloc[0]
#     df = pd.read_csv(feats_csv_path)
#     feats = shuffle(df).reset_index(drop=True)
#     feats = feats.to_numpy()
#     label = np.zeros(args.num_classes)
#     if args.num_classes==1:
#         label[0] = csv_file_df.iloc[1]
#     else:
#         if int(csv_file_df.iloc[1])<=(len(label)-1):
#             label[int(csv_file_df.iloc[1])] = 1
#     labels = [feature[f'index{index_dict[i]}'][1] for i in indexes]  #list of labels, in form of ['H','L','L'...]
#     feats = [feature[f'index{index_dict[i]}'][2] for i in indexes]
    
    global features
    global index_dict
    
    feats_og = pd.DataFrame(features[f'index{index_dict[index]}'][2])
    feats = shuffle(feats_og).reset_index(drop=True).to_numpy()
    label_og = label_dict[features[f'index{index_dict[index]}'][1]]   # transformed label in form of int,[0,1]
    
    label = np.zeros(args.num_classes)
    if args.num_classes==1:
        label[0] = label_og
    else:
        if int(label_og)<=(len(label)-1):
            label[int(label_og)] = 1
    return label,feats

In [98]:
_,f = get_bag_feats(test_index[0],args)

In [99]:
f

array([[ 0.03614862,  0.01931491,  0.00420622, ..., -0.03121399,
        -0.02962754,  0.02326941],
       [-0.01056455,  0.00552965,  0.00076318, ...,  0.00204356,
        -0.00944971,  0.03375618],
       [ 0.01624807, -0.02702549,  0.03657382, ...,  0.00324069,
        -0.0203822 ,  0.00076802],
       ...,
       [ 0.01556042, -0.01698582,  0.00343989, ..., -0.00370609,
         0.01280652, -0.03815655],
       [-0.02748846, -0.00973979,  0.00167668, ...,  0.01203365,
        -0.0328464 , -0.00955074],
       [ 0.02604579, -0.03191727, -0.07271944, ..., -0.01442208,
        -0.00617436,  0.07784676]], dtype=float32)